In [157]:
import pandas as pd
import numpy as np
import math
from unicodedata import category
df=pd.read_csv("Train_B_Bayesian.csv")
def normal(x,mu,sigma):
    return (1/(math.sqrt(2*np.pi)*sigma))*np.exp(-(((x-mu)/sigma)**2)/2)

In [158]:
def Bayes(train, val, test, flag):
    posframe=train[train['is_patient']==1]
    negframe=train.drop(posframe.index)
    P={}
    N={}
    for i in train.columns[:-2]:
        P[i]={}
        N[i]={}
        if i=='gender':
            P[i][0]=len(posframe[posframe[i]==0])/len(train)
            N[i][0]=len(negframe[negframe[i]==0])/len(train)
            P[i][1]=len(posframe[posframe[i]==1])/len(train)
            N[i][1]=len(negframe[negframe[i]==1])/len(train)
        else:
            P[i]['mean']=posframe[i].mean()
            P[i]['std']=posframe[i].std()
            N[i]['mean']=negframe[i].mean()
            N[i]['std']=negframe[i].std()
    if(not flag):
        corr=0
        for i in val.index:
            p_prob = 1
            n_prob = 1
            for j in val.columns[1:-2]:
                if j=='gender':
                    p_prob*=P[j][val[j][i]]
                    n_prob*=N[j][val[j][i]]
                else:
                    p_prob*=normal(val[j][i],P[j]['mean'],P[j]['std'])*len(posframe[posframe[j]==val[j][i]])/len(test)
                    n_prob*=normal(val[j][i],N[j]['mean'],N[j]['std'])*len(negframe[negframe[j]==val[j][i]])/len(test)
            if(p_prob>=n_prob):
                pred=1
            else:
                pred=2
            if(val['is_patient'][i]==pred):
                corr+=1
        return corr/len(val)
    else:
        corr=0
        for i in test.index:
            p_prob = 1
            n_prob = 1
            for j in test.columns[1:-2]:
                if j=='gender':
                    p_prob*=P[j][test[j][i]]
                    n_prob*=N[j][test[j][i]]
                else:
                    p_prob*=normal(test[j][i],P[j]['mean'],P[j]['std'])*len(posframe[posframe[j]==test[j][i]])/len(test)
                    n_prob*=normal(test[j][i],N[j]['mean'],N[j]['std'])*len(negframe[negframe[j]==test[j][i]])/len(test)
            if(p_prob>=n_prob):
                pred=1
            else:
                pred=2
            if(test['is_patient'][i]==pred):
                corr+=1
        return corr/len(test)

In [159]:
df

,age,gender,tot_bilirubin,direct_bilirubin,tot_proteins,albumin,ag_ratio,sgpt,sgot,alkphos,is_patient
0,65,Female,0.7,0.1,187,16,18,6.8,3.3,0.90,1
1,62,Male,10.9,5.5,699,64,100,7.5,3.2,0.74,1
2,62,Male,7.3,4.1,490,60,68,7.0,3.3,0.89,1
3,58,Male,1.0,0.4,182,14,20,6.8,3.4,1.00,1
4,72,Male,3.9,2.0,195,27,59,7.3,2.4,0.40,1
...,...,...,...,...,...,...,...,...,...,...,...
578,60,Male,0.5,0.1,500,20,34,5.9,1.6,0.37,2
579,40,Male,0.6,0.1,98,35,31,6.0,3.2,1.10,1
580,52,Male,0.8,0.2,245,48,49,6.4,3.2,1.00,1
581,31,Male,1.3,0.5,184,29,32,6.8,3.4,1.00,1


In [160]:
S = df.isnull().sum()
S = S[S!=0]
for i in S.index:
    m = df[i].mean()
    for j in df.index:
        if(np.isnan(df[i][j])):
            df.loc[j,i]=m
            print(i,j,df[i][j])
MAX_OC=0
for j in df.index:
    df.loc[j,'OC']=0
for i in df.columns[:-2]:
    if(df[i].dtype=='object'):
        df[i] = df[i].astype('category').cat.codes
    m=df[i].mean()
    s=df[i].std()
    for j in df.index:
        if(df[i][j]>2*m+5*s):
            df.loc[j,'OC'] += 1
            if(df.loc[j,'OC']>MAX_OC):
                MAX_OC=df.loc[j,'OC']
print(MAX_OC)
for j in df.index:
    if(df.loc[j,'OC']==MAX_OC):
        df=df.drop(j)


alkphos 209 0.9470639032815197
alkphos 241 0.9470639032815197
alkphos 253 0.9470639032815197
alkphos 312 0.9470639032815197
2.0


In [161]:
df

,age,gender,tot_bilirubin,direct_bilirubin,tot_proteins,albumin,ag_ratio,sgpt,sgot,alkphos,is_patient,OC
0,65,0,0.7,0.1,187,16,18,6.8,3.3,0.90,1,0.0
1,62,1,10.9,5.5,699,64,100,7.5,3.2,0.74,1,0.0
2,62,1,7.3,4.1,490,60,68,7.0,3.3,0.89,1,0.0
3,58,1,1.0,0.4,182,14,20,6.8,3.4,1.00,1,0.0
4,72,1,3.9,2.0,195,27,59,7.3,2.4,0.40,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
578,60,1,0.5,0.1,500,20,34,5.9,1.6,0.37,2,0.0
579,40,1,0.6,0.1,98,35,31,6.0,3.2,1.10,1,0.0
580,52,1,0.8,0.2,245,48,49,6.4,3.2,1.00,1,0.0
581,31,1,1.3,0.5,184,29,32,6.8,3.4,1.00,1,0.0


In [162]:
train=df.sample(frac=0.7)
test=df.drop(train.index)
test=test.reset_index()
T=np.array_split(train,5)
acc=[]
for i in range(5):
    train_5 = train.drop(T[i].index)
    acc.append(Bayes(train_5,T[i],test,0))
acc=np.array(acc)
I,max_acc=np.argmax(acc),np.max(acc)
print("\033[3;36m"+str(I)+"\033[0;m\t\033[1;34m"+str(max_acc)+"\033[0;m")
train_5 = train.drop(T[I].index)
print("\033[3;92m"+str(Bayes(train_5,T[I],test,1)))

3	0.7530864197530864
0.7471264367816092
